In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import grangercausalitytests
import warnings
from sklearn.tree import DecisionTreeRegressor
from statsmodels.tsa.seasonal import seasonal_decompose
from pylab import rcParams
from sklearn.metrics import mean_squared_error
import warnings
from dateutil.relativedelta import relativedelta
from datetime import datetime
warnings.filterwarnings("ignore")
%matplotlib inline

## Data extraction

Extracting data from database and Excel files.
<br/><br/>
1. **Weather data**
    - Source: OpenMeteo
    - Database location: gold.v_openmeteo_station_daily
    - Frequency: Daily
    - Country: Malaysia
<br/><br/>
2. **Fertilizers data** - we are using sum of potash imports from different countries to Malaysia.
    - Source: TDM 
    - Database location: gold.tbl_tradedata
    - Frequency: Monthly
    - Country: Malaysia
<br/><br/>
3. **Palm Oil Production data** 
    - Source: Data published by Malaysian government. \
        We store this data in SnD file:  
        "G:\Shared drives\1-Commodity Markets Analysis\2-Commodities\4 - Oils\SnD\Fats - SnD - Azure connection.xlsm"
    - Country: Malaysia

In [ ]:
df_weather = spark.sql("""
    SELECT * 
    FROM gold.v_openmeteo_station_daily 
    WHERE CountryIso2 = 'MY' 
    AND StationName IN (
        'LABUAN', 'KOTA KINABALU INTL', 'SANDAKAN', 'TAWAU', 'MIRI', 
        'BINTULU', 'SIBU', 'KUCHING INTL', 'SULTAN ISMAIL', 'MERSING', 
        'MALACCA', 'SINGAPORE CHANGI INTL', 'KUANTAN'
    )
""")
df_weather = df_weather.toPandas()

'''
df_potash = spark.sql("""
SELECT * 
FROM gold.v_tradedata 
WHERE Scope ='TropicalOils'
AND Flow ='I'
AND Commodity = 'POT' 
AND Reporter = 'MYS'
AND Type = 'R'
""")
df_potash = df_potash.toPandas()
'''

df_potash = pd.read_csv('Potash_imports_prd.csv')

df_production = pd.read_excel('Oil_Production_Raw_Data.xlsx')

df_area = pd.read_excel('Oil_Production_Raw_Data.xlsx', sheet_name = 'Area')

In [38]:
regions = [
    df_weather['StationName'].isin(['LABUAN', 'KOTA KINABALU INTL', 'SANDAKAN', 'TAWAU']),
    df_weather['StationName'].isin(['MIRI', 'BINTULU', 'SIBU', 'KUCHING INTL']),
    df_weather['StationName'].isin(['SULTAN ISMAIL', 'MERSING', 'MALACCA', 'SINGAPORE CHANGI INTL']),
    df_weather['StationName'] == 'KUANTAN'
]
choices = ['SABAH', 'SARAWAK', 'JOHOR', 'PAHANG']

df_weather['StationRegion'] = np.select(regions, choices, default='UNKNOWN')

In [39]:
df_potash.head()

In [40]:
df_production.head()

In [41]:
df_area.head()

### Pivot of weather table

Creating table where we have column with date and columns with weather data for each station, ex. 'Temperature_2m_max_BINTULU'.

In [42]:
df_weather_pivot = df_weather.pivot_table(
    index='Event_date',
    columns='StationName',
    values=[
        'Precipitation_sum', 
        'Temperature_2m_max', 
        'Temperature_2m_min', 
        'Temperature_2m_avg', 
        'Sunshine_duration', 
        'Wind_speed_10m_max', 
        'Et0_fao_evapotranspiration'
    ],
    aggfunc='mean'
)

In [43]:
# Define the aggregation functions for each column
agg_funcs = {
    'Precipitation_sum': 'sum',
    'Temperature_2m_max': 'max',
    'Temperature_2m_min': 'min',
    'Temperature_2m_avg': 'mean',
    'Sunshine_duration': 'mean',
    'Wind_speed_10m_max': 'mean',
    'Et0_fao_evapotranspiration': 'mean'
}

# Create the pivot table
df_weather_pivot = df_weather.pivot_table(
    index='Event_date',
    columns='StationRegion',
    values=list(agg_funcs.keys()),
    aggfunc=agg_funcs
)

# Reset the column names to avoid MultiIndex columns
df_weather_pivot.columns = ['_'.join(col).strip() for col in df_weather_pivot.columns.values]
df_weather_pivot.reset_index(inplace=True)

# Display the resulting pivot table
df_weather_pivot

In [44]:
df_weather_pivot['Event_date'] = pd.to_datetime(df_weather_pivot['Event_date'])
df_weather_pivot.set_index('Event_date', inplace=True)

In [45]:
# Define the aggregation rules
def aggregation_rules(column_name):
    if column_name.startswith('Precipitation_sum'):
        return 'sum'
    if column_name.startswith('Temperature_2m_min'):
        return 'min'
    if column_name.startswith('Temperature_2m_max'):
        return 'max'
    else:
        return 'mean'

# Prepare the aggregation dictionary
agg_rules = {}
for col in df_weather_pivot.columns:
    rule = aggregation_rules(col)
    if isinstance(rule, list):  # If multiple aggregations, expand them
        agg_rules[col] = rule
    else:
        agg_rules[col] = rule

# Apply the aggregation
df_weather_monthly = df_weather_pivot.resample('M').agg(agg_rules)

# Flatten the resulting MultiIndex columns
df_weather_monthly.columns = [
    f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col for col in df_weather_monthly.columns
]
df_weather_monthly

In [46]:
df_weather_pivot.columns

In [47]:
monthly_temperature_thresholds = df_weather_pivot.groupby(pd.Grouper(freq='M')).agg(
    Days_temp_above_SARAWAK=('Temperature_2m_max_SARAWAK', lambda x: (x > 32).sum()),
    Days_temp_below_SARAWAK=('Temperature_2m_min_SARAWAK', lambda x: (x < 22).sum()),
    Days_temp_above_JOHOR=('Temperature_2m_max_JOHOR', lambda x: (x > 32).sum()),
    Days_temp_below_JOHOR=('Temperature_2m_min_JOHOR', lambda x: (x < 22).sum()),
    Days_temp_above_PAHANG=('Temperature_2m_max_PAHANG', lambda x: (x > 32).sum()),
    Days_temp_below_PAHANG=('Temperature_2m_min_PAHANG', lambda x: (x < 22).sum()),
    Days_temp_above_SABAH=('Temperature_2m_max_SABAH', lambda x: (x > 32).sum()),
    Days_temp_below_SABAH=('Temperature_2m_min_SABAH', lambda x: (x < 22).sum()),
    Max_temp_SARAWAK=('Temperature_2m_max_SARAWAK', 'max'),
    Min_temp_SARAWAK=('Temperature_2m_min_SARAWAK', 'min'),
    Max_temp_JOHOR=('Temperature_2m_max_JOHOR', 'max'),
    Min_temp_JOHOR=('Temperature_2m_min_JOHOR', 'min'),
    Max_temp_PAHANG=('Temperature_2m_max_PAHANG', 'max'),
    Min_temp_PAHANG=('Temperature_2m_min_PAHANG', 'min'),
    Max_temp_SABAH=('Temperature_2m_max_SABAH', 'max'),
    Min_temp_SABAH=('Temperature_2m_min_SABAH', 'min'),
).reset_index()

monthly_temperature_thresholds['Amplitude_SARAWAK'] = monthly_temperature_thresholds['Max_temp_SARAWAK'] - monthly_temperature_thresholds['Min_temp_SARAWAK']
monthly_temperature_thresholds['Amplitude_JOHOR'] = monthly_temperature_thresholds['Max_temp_JOHOR'] - monthly_temperature_thresholds['Min_temp_JOHOR']
monthly_temperature_thresholds['Amplitude_PAHANG'] = monthly_temperature_thresholds['Max_temp_PAHANG'] - monthly_temperature_thresholds['Min_temp_PAHANG']
monthly_temperature_thresholds['Amplitude_SABAH'] = monthly_temperature_thresholds['Max_temp_SABAH'] - monthly_temperature_thresholds['Min_temp_SABAH']

monthly_temperature_thresholds = monthly_temperature_thresholds.drop(['Max_temp_SARAWAK', 'Min_temp_SARAWAK', 'Max_temp_JOHOR', 'Min_temp_JOHOR', 'Max_temp_PAHANG', 'Min_temp_PAHANG', 'Max_temp_SABAH', 'Min_temp_SABAH'], axis = 1)

monthly_temperature_thresholds.set_index('Event_date', inplace=True)
monthly_temperature_thresholds

In [48]:
df_weather_monthly = pd.merge(df_weather_monthly, monthly_temperature_thresholds, left_index=True, right_index=True, how='left')

In [49]:
df_weather_monthly.index = pd.to_datetime(df_weather_monthly.index)
df_weather_monthly.index = df_weather_monthly.index.to_period('M').to_timestamp()

In [50]:
df_weather_monthly.head()

In [51]:
df_potash

In [52]:
df_potash.columns

In [53]:
df_potash = df_potash.drop(['Flow', 'Year', 'Month', 'Seasonal', 'Commodity', 'Reporter',
       'Partner', 'Unit', 'Cy', 'Value_Per_Unit', 'Value', 'Scope',
       'Type'], axis = 1)

In [54]:
df_potash.head()

In [55]:
df_potash = df_potash.groupby('Date', as_index=False)['Qty'].sum()
df_potash.head()

In [56]:
df_potash['Date'] = pd.to_datetime(df_potash['Date'])
df_potash.set_index('Date', inplace=True)
df_potash

In [57]:
df_area['Date'] = pd.to_datetime(df_area['Date'])
df_area.set_index('Date', inplace=True)
df_area

In [58]:
df_potash = pd.merge(df_potash, df_area, left_index=True, right_index=True, how='left')

In [59]:
df_potash['Qty_kg/ha'] = df_potash['Qty']*1000/df_potash['Area']

In [60]:
df_potash

In [61]:
plt.plot(df_potash.index, df_potash['Qty_kg/ha'])

In [62]:
df_production['Date'] = pd.to_datetime(df_production['Date'])
df_production.set_index('Date', inplace=True)

In [73]:
plt.figure(figsize=(10, 5))
plt.plot(df_production.index, df_production['Palm Oil'])

In [64]:
df = pd.merge(df_production, df_potash['Qty_kg/ha'], left_index=True, right_index=True, how='left')  # Use 'outer', 'left', or 'right' as needed
df = pd.merge(df, df_weather_monthly, left_index=True, right_index=True, how='left')

In [65]:
df

In [66]:
# Mapping for replacements
replacements = {
    'Palm Oil': 'Palm_Oil',
    'Qty': 'Potash_Qty',
    'Et0_fao_evapotranspiration_': 'Evapo_',
    'Precipitation_': 'Precip_',
    'Sunshine_duration_': 'Sunshine_',
    'Temperature_2m_avg_': 'Temp_avg_',
    'Temperature_2m_max_': 'Temp_max_',
    'Temperature_2m_min_': 'Temp_min_',
    'Wind_speed_10m_max_': 'Wind_speed_',
    'Days_temp_below_': "Days_temp_below_",
    'Days_temp_above_': "Days_temp_above_",
    'Amplitude': 'Amplitude'
}

# Correct renaming logic
simplified_columns = []
for col in df.columns:
    new_col = col
    for old, new in replacements.items():
        if old in new_col:
            new_col = new_col.replace(old, new)
    simplified_columns.append(new_col)

# Apply simplified column names
df.columns = simplified_columns

In [67]:
df.columns

In [68]:
# Define weights for the regions
weights = {
    "JOHOR": 0.30,
    "PAHANG": 0.20,
    "SABAH": 0.25,
    "SARAWAK": 0.25
}

weights = {key: float(value) for key, value in weights.items()}

# List of weather-related columns to compute weighted averages
weather_columns = [
    "Evapo",
    "Precip_sum",
    "Temp_min",
    "Temp_max",
    "Wind_speed",
    "Sunshine",
    "Temp_avg",
    "Days_temp_below",
    "Days_temp_above",
    "Amplitude"
]

for col in weather_columns:
    for region in ["JOHOR", "PAHANG", "SABAH", "SARAWAK"]:
        df[f"{col}_{region}"] = df[f"{col}_{region}"].astype(float)

# Calculate weighted average for each weather-related parameter
for col in weather_columns:
    weighted_col_name = f"{col}_weighted_avg"
    df[weighted_col_name] = (
        df[f"{col}_JOHOR"] * weights["JOHOR"] +
        df[f"{col}_PAHANG"] * weights["PAHANG"] +
        df[f"{col}_SABAH"] * weights["SABAH"] +
        df[f"{col}_SARAWAK"] * weights["SARAWAK"]
    )